Import data needed for part 1 

In [1]:
import pandas as pd


In [8]:
data_path = "./Q1_Sample_Data.csv"

data_raw = pd.read_csv(data_path)
data_raw.shape

(3553, 4)

In [3]:
data_raw.head()

,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG
4,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G


In [4]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3553 entries, 0 to 3552
Data columns (total 4 columns):
ts            3553 non-null object
user_id       3553 non-null object
country_id    3553 non-null object
site_id       3553 non-null object
dtypes: object(4)
memory usage: 111.1+ KB


In [7]:
data_raw.isna().sum()

ts            0
user_id       0
country_id    0
site_id       0
dtype: int64

In [64]:
data_raw.describe()

,ts,user_id,country_id,site_id
count,3553,3553,3553,3553
unique,3538,1916,7,8
top,2019-02-03 20:00:34,LC3A59,TL6,5NPAU
freq,3,149,1449,1614


### Question 1 

Consider only the rows with country_id = "BDV" (there are 844 such rows).  
For each site_id, we can compute the number of unique user_id's found in these 844 rows.  
Which site_id has the largest number of unique users? And what's the number?

In [16]:
q1_output = (data_raw
             .loc[lambda x: x.country_id == 'BDV']
             .groupby('site_id')
             .agg({'user_id' : 'nunique' })
            )

In [17]:
q1_output

,user_id
site_id,
3POLC,2
5NPAU,544
N0OTG,90


In [22]:
print(f'site_id with largest number of unique users: {q1_output.idxmax()[0]} with {q1_output.max()[0]} unique users')

site_id with largest number of unique users: 5NPAU with 544 unique users


In [14]:
q1_output.describe()

,ts,user_id,country_id,site_id
count,844,844,844,844
unique,842,623,1,3
top,2019-02-06 12:41:15,LCC346,BDV,5NPAU
freq,2,15,844,717


### Question 2 

Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times.  
Find these four users & which sites they (each) visited more than 10 times.  
(Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.)

In [23]:
# First I will start by converting the ts column to a date time format 

data = (data_raw
        .assign(ts = lambda x: pd.to_datetime(x.ts))
       )

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3553 entries, 0 to 3552
Data columns (total 4 columns):
ts            3553 non-null datetime64[ns]
user_id       3553 non-null object
country_id    3553 non-null object
site_id       3553 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 111.1+ KB


In [26]:
time_limit_inf = pd.to_datetime('2019-02-03 00:00:00')
time_limit_sup = pd.to_datetime('2019-02-04 23:59:59')

In [33]:
q2_output = (data
             .loc[lambda x : (x.ts >= time_limit_inf) & (x.ts <= time_limit_sup)] # slicing according to timestamp
             .groupby(['user_id', 'site_id'], as_index = False) # grouping by unique user/site pairs
             .agg({'ts':'count'}) # counting the number of visit for each pair
             .loc[lambda x: x.ts > 10] # selecting pairs above 10 visits
            )

In [51]:
q2_output

,user_id,site_id,ts
3,LC06C3,N0OTG,25
417,LC3A59,N0OTG,26
485,LC3C7E,3POLC,15
493,LC3C9D,N0OTG,17


In [50]:
print(q2_output.values)

[['LC06C3' 'N0OTG' 25]
 ['LC3A59' 'N0OTG' 26]
 ['LC3C7E' '3POLC' 15]
 ['LC3C9D' 'N0OTG' 17]]


### Question 3 

For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. Based on this measure, what are top three sites? (hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; simply provide three pairs in the form (site_id, number of users).

In [67]:
q3_output = (data
             .groupby('user_id', as_index = False) # first I select the last visit for each user 
             .agg({'site_id':'last'}) # there are now only 1 visit by user (the last)
             .groupby('site_id') # then I count the number of user for each site
             .agg({'user_id':'count'})
             .sort_values('user_id', ascending=False) # sorting in descending order
            )

In [68]:
q3_output.head(3)

,user_id
site_id,
5NPAU,992
N0OTG,561
QGO3G,289


In [63]:
# check the hint 
q3_output.head()

,user_id
site_id,
5NPAU,992
N0OTG,561
QGO3G,289
GVOFK,42
3POLC,28


In [66]:
# check sum 
(q3_output.sum() == data.user_id.nunique())

user_id    True
dtype: bool

### Question 4 

For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. Compute the number of users whose first/last visits are to the same website. What is the number?

In [82]:
# using a similar method to the previous question, I start by identifying the first and last site visited for each user 
# Then, I compute the number of user with matching first and last visits

q4_output = (data
             .groupby('user_id', as_index = False)
             .agg({'site_id':['first','last']}) 
             .loc[lambda x: x.site_id['first'] == x.site_id['last']]
             .shape[0]
            )

In [86]:
f'There are {q4_output} user whose first/last visits are to the same site, including user with a unique visit'

'There are 1670 user whose first/last visits are to the same site, including user with a unique visit'

In [90]:
# We can also consider only user with more than one visit (so first and last are two different visits)

q4_output_alternate = (data
                       .groupby('user_id', as_index = False)
                       .agg({'site_id':['first','last'], 'ts': 'count'})
                       .loc[lambda x: (x.site_id['first'] == x.site_id['last']) & (x.ts['count'] > 1)]
                       .shape[0]
                      )

In [92]:
f'When excluding user with only one visit on record, there are {q4_output_alternate} users with matching first and last visits'

'When excluding user with only one visit on record, there are 409 users with matching first and last visits'